In [1]:
import numpy as np
from matplotlib import pyplot as plt
import h5py

import itertools
from sys import getsizeof


In [2]:
def relu(x):
    if x < 0:
        return 0
    else:
        return x
        

In [3]:
weight_range = np.arange(0, 1.5, 0.1)
input_range = [1]

# constraints
# w_ee = w_ll

param_grid = {
            'w_ll': weight_range,
            'w_le': weight_range,
            'w_li': weight_range,
            'w_el': weight_range,
            # 'w_ee': weight_range,
            'w_ei': weight_range,
            'w_il': weight_range,
            # 'w_ie': weight_range,
            # 'w_ii': weight_range,
            's_l': input_range,
            's_e': input_range,
            's_i': input_range,
}

keys = list(param_grid.keys())
values = list(param_grid.values())

all_param_combinations = list(itertools.product(*values))

print("Length = ", len(all_param_combinations))
print("Memory allocated = ", getsizeof(all_param_combinations)/1e6) # MB


Length =  11390625
Memory allocated =  100.232088


when the weight range for each weight is about 10 values (0 to 1, at a step of 0.1), this leads to 1 billion combinations of just weights, and 1 trillion combinations of weights + input values, which is very high to even store as combinations.
as long as the params dictionary has 7 independent variables (leading to 10 million combinations, its handelable). The size is about 90 MB. For 8 independent variables the size is 900 MB (of just the list containing the combinations). And for 9 variables it must be about 9 GB.

## infer the solution space for early recall

In [4]:
stead_state_activity = [1, 0, 1]

results = []

for params in all_param_combinations:

    param_dict = dict(zip(keys, params))

    w_ee = param_dict['w_ll']

    W = np.array([
        [param_dict['w_ll'], param_dict['w_le'], - param_dict['w_li']],
        [param_dict['w_el'], w_ee, - param_dict['w_ei']],
        [param_dict['w_il'], param_dict['w_ie'], - param_dict['w_ii']]
    ])

    s = np.array([param_dict['s_l'], param_dict['s_e'], param_dict['s_i']])
    I = np.eye(3)

    try:
        r_ss = np.linalg.solve(I-W, s)
    except np.linalg.LinAlgError:
        r_ss = np.full(3, np.nan)

    results.append({'params': param_dict, 'steady_state': r_ss})

# After your simulation loop:
params_array = np.array([list(r['params'].values()) for r in results])
steady_states_array = np.array([r['steady_state'] for r in results])

with h5py.File('simulation_results.h5', 'w') as f:
    f.create_dataset('params', data=params_array, compression='gzip')
    f.create_dataset('steady_states', data=steady_states_array, compression='gzip')
    f.create_dataset('param_grid', data=param_grid, compression='gzip')

KeyError: 'w_ie'